## EDA of combined data.

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings(action='ignore')

In [6]:
#readings the datasets
cust_add = pd.read_csv('CustomerAddress.csv')
cust_demo = pd.read_csv('CustomerDemographic.csv')
newcust_list = pd.read_csv('NewCustomerList_cleaned.csv')
transactions = pd.read_csv('transactions_cleaned_csv')

In [20]:
cust_add.head(2)

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,NSW,Australia,10
1,2,6 Meadow Vale Court,2153,NSW,Australia,10


In [21]:
cust_demo.head(2)

,customer_id,first_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,Laraine,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,71
1,2,Eli,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,44


In [10]:
print(cust_add.shape)
cust_demo.shape

(3999, 6)


(3999, 12)

* Joining the `CustomerAddress` and `CustomerDemographics` using **customer_id**.

In [22]:
cust_data = pd.merge(cust_demo, cust_add, how='inner', on='customer_id')
cust_data.head(2)

,customer_id,first_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age,address,postcode,state,country,property_valuation
0,1,Laraine,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,71,060 Morning Avenue,2016,NSW,Australia,10
1,2,Eli,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,44,6 Meadow Vale Court,2153,NSW,Australia,10


In [18]:
cust_data.shape

(3995, 17)

In [23]:
newcust_list.head(2)

,first_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,Age,Age_category
0,Chickie,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.71875,67,Old
1,Morly,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.71875,54,Adult


* Since the NewCustomerList lacks a customer ID column, concatenating it with the old customer data won't work because the columns aren't identical.
* Therefore, we'll proceed with the RFM analysis using the `cust_data` table and transactions.
* After completing the analysis, we can compare the old and new customer datasets

In [24]:
# merging transaction and cust_data tables
cmp_data = pd.merge(transactions, cust_data, left_on='customer_id', right_on='customer_id', how='inner')
cmp_data.shape

(19959, 30)

In [25]:
cmp_data.head(3)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,wealth_segment,deceased_indicator,owns_car,tenure,Age,address,postcode,state,country,property_valuation
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,...,Mass Customer,N,Yes,10.0,68,984 Hoepker Court,3064,VIC,Australia,6
1,11065,1,2950,2017-10-16,0.0,Approved,Giant Bicycles,Standard,medium,medium,...,Mass Customer,N,Yes,10.0,68,984 Hoepker Court,3064,VIC,Australia,6
2,18923,62,2950,2017-04-26,0.0,Approved,Solex,Standard,medium,medium,...,Mass Customer,N,Yes,10.0,68,984 Hoepker Court,3064,VIC,Australia,6


* We now have all the data combined into a single table called cmp_data, which will be used for EDA and then for RFM analysis.

### Exploratory Data Analysis

In [26]:
cmp_data.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date', 'Profit', 'first_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'Age', 'address', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object')